In [1]:
import sys
import os
import json

sys.path.append("../../")

import datetime
from typing import Dict, List

from pipeline.extract import (
    extract_intent_upvotes,
    extract_intents,
    extract_interests,
    extract_match_stars,
    extract_recent_matches,
    extract_users,
)

from pipeline.transform import (
    augment_users_with_intents,
    augment_users_with_interests,
    compute_internal_matching_metrics,
    convert_matches_from_df,
    convert_users_from_df,
    filter_recent_matches,
    get_matching_input,
    transform_matches_for_load,
)

from pipeline.utils.firebase import initialize_firebase_for_prefect

db = initialize_firebase_for_prefect.run(os.environ['VITE_firebase_databaseURL'],os.environ['API_ADMIN_CREDENTIALS'])
param_community = 'sds'

In [3]:
df_users = extract_users.run(db, param_community)

[2022-07-22 00:34:25+0000] INFO - prefect | Returned 25 rows, 5 cols.


In [ ]:
df_users

In [5]:
raw_intents = extract_intents.run(db, param_community)
raw_interests = extract_interests.run(db, param_community)

users_w_profile = convert_users_from_df.run(df_users)
users_w_intents = augment_users_with_intents.run(
    users_w_profile, raw_intents
)
users_w_data = augment_users_with_interests.run(
    users_w_intents, raw_interests
)

In [16]:
users = users_w_data

In [20]:
from pipeline.types import (
    Intent,
    IntentMatch,
    Match,
    MatchingInput,
    MatchMetadata,
    Side,
    User,
)

from typing import Dict, Iterator

def get_lookup(u1, u2):
    return f"{u1},{u2}" if u1 < u2 else f"{u2},{u1}"

all_matches: Dict[str, Match] = {}

In [ ]:
        for idx, u in enumerate(users):
            # create Intent info from the current user
            intents = u.intents

            # users can have more than one intent, this loops through each one
            for intent in intents:
                code = intent.code
                side = intent.side
                name = intent.name

                # otherSide would be the other side of the intent that they have
                other_side = (
                    Side.GIVING if side == Side.SEEKING else Side.SEEKING
                )

                # create an Intent object to search for in the other users
                otherIntent = Intent(code, other_side, name)

                for oth in users:
                    if otherIntent in oth.intents:

                        # create the intent match after finding a user that matches
                        intent_match = IntentMatch(
                            code,
                            seeker=oth.uid
                            if otherIntent.side == Side.SEEKING
                            else u.uid,
                            giver=oth.uid
                            if otherIntent.side == Side.GIVING
                            else u.uid,
                        )

                        # key contains the user ids that have matched
                        key = get_lookup(u.uid, oth.uid)
                        # checks if the users have been matched, prevents duplicates
                        if key in all_matches:
                            # additional check if the users have NOT matched on this intent and appends it to the matchingIntents
                            if (
                                intent_match
                                not in all_matches[key].metadata.matchingIntents
                            ):
                                all_matches[
                                    key
                                ].metadata.matchingIntents.append(intent_match)
                        else:
                            new_match = Match(
                                users={u.uid, oth.uid},
                                metadata=MatchMetadata(
                                    generator="GENERATOR_SIMILAR_INTENTS",
                                    score=1,
                                    matchingIntents=[intent_match],
                                ),
                            )
                            all_matches[key] = new_match

        # all matches are stored in the dictionary as values

In [ ]:
print(all_matches["DunEgGqeSfaWQYskRFuvlzYceYB3,DunEgGqeSfaWQYskRFuvlzYceYB3"])

In [26]:
def converts_user_data_to_vector(users_w_data): # return pandas dataframe
    # have 3 bits for 1 category
    # 3 * 2 = 6 bits
    # first 2 bits are seeker or receiver for careers informations -> "01", "10", "11"
    # next we look at movie recs -> "1001"
    print(users_w_data)


In [27]:
converts_user_data_to_vector(users_w_data)

[User(uid='AmkGRnFOlKaqTzU4yiXqRMA0GRf2', displayName='Grace Sopha', interests=[], intents=[], schedule=[]), User(uid='DunEgGqeSfaWQYskRFuvlzYceYB3', displayName='Lauren Williams', interests=[], intents=[Intent(code='tech-careers', side=<Side.GIVING: 'giving'>, name='tech-careers'), Intent(code='tech-careers', side=<Side.SEEKING: 'seeking'>, name='tech-careers')], schedule=[]), User(uid='HnEWnuWHXXf4X99Orq6eC3CC5GT2', displayName='Neveen Elmajdoub', interests=[], intents=[], schedule=[]), User(uid='JpsfDVkhxSdWZz4v1OEMm1U1Agk1', displayName='Christopher Hunter', interests=[], intents=[], schedule=[]), User(uid='KQSpHWoyZHYZ1pvuk5Y046IG8xf2', displayName='Luke Arrington', interests=[], intents=[], schedule=[]), User(uid='Ks2YhmRXOGdUvPJwWaO9gRh29l73', displayName='Emma Diamon', interests=[], intents=[Intent(code='involve-iit', side=<Side.GIVING: 'giving'>, name='involve-iit'), Intent(code='involve-iit', side=<Side.SEEKING: 'seeking'>, name='involve-iit'), Intent(code='tech-careers', sid